# Esta tarea fue realizada en equipo por Betsy Torres, Mariana Briones & Erick Mendoza

# Tarea 3.2

Usando la propiedades de variables aelatorias y el archivo de las ventas de cada año, responda analíticamente.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web 
import warnings
from scipy.stats import norm

In [21]:
def tabla_resumen(util):
    '''se inserta la tabla con los flujos'''
    vec_datos_irr = np.zeros(util.shape[0],)
    vec_datos_vpn=np.zeros(util.shape[0],)
    for i in range(len(vec_datos_irr)):
        vec_datos_irr[i] = np.irr(util.loc[i].values)
        vec_datos_vpn[i] = np.npv(rf,util.loc[i].values)
        util["IRR"]=vec_datos_irr*100
        util["VPN"]=vec_datos_vpn
    return(util)

def func(rf:'Tasa libre de riesgo', flujos:'flujos de efectivo'):
    '''Esta función calculará la TIE y el VPN de los datos, los cuales son fujos 
    de efectivo, y arrojará los resultados por medio de listas'''
    l_irr=[]
    l_npv=[]

    for i in range(len(flujos)):
        ans_irr=np.irr(flujos.iloc[i,0:6])*100
        ans_npv=np.npv(rf,flujos.iloc[i,0:6])
        
        l_irr.append(ans_irr)        
        l_npv.append(ans_npv)
    
    return l_irr,l_npv

# Create models from data
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Distributions to check
   # DISTRIBUTIONS = [        
      #  st.alpha,st.anglit,st.arcsine,st.beta,st.betaprime,st.bradford,st.burr,st.cauchy,st.chi,st.chi2,st.cosine,
       # st.dgamma,st.dweibull,st.erlang,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,st.fatiguelife,st.fisk,
        #st.foldcauchy,st.foldnorm,st.frechet_r,st.frechet_l,st.genlogistic,st.genpareto,st.gennorm,st.genexpon,
      #  st.genextreme,st.gausshyper,st.gamma,st.gengamma,st.genhalflogistic,st.gilbrat,st.gompertz,st.gumbel_r,
      #  st.gumbel_l,st.halfcauchy,st.halflogistic,st.halfnorm,st.halfgennorm,st.hypsecant,st.invgamma,st.invgauss,
      #  st.invweibull,st.johnsonsb,st.johnsonsu,st.ksone,st.kstwobign,st.laplace,st.levy,st.levy_l,st.levy_stable,
      #  st.logistic,st.loggamma,st.loglaplace,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,
      #  st.nct,st.norm,st.pareto,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,st.rdist,st.reciprocal,
      #  st.rayleigh,st.rice,st.recipinvgauss,st.semicircular,st.t,st.triang,st.truncexpon,st.truncnorm,st.tukeylambda,
      #  st.uniform,st.vonmises,st.vonmises_line,st.wald,st.weibull_min,st.weibull_max,st.wrapcauchy
    #]

  # Distributions to check
    DISTRIBUTIONS = [        
        
        st.genexpon,st.lognorm,st.maxwell,st.ncx2,
        
        st.norm, st.uniform,st.beta #agregar Beta
    ]


    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return (best_distribution.name, best_params)

def make_pdf(dist, params, size=10000):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

**a)¿Cual es el valor esperado de las ventas del año 1 y año 2?**

El año 1 sigue una distribución normal y el año dos sigue una distribución uniforme, de manera que para una distribución normal, el valor esperado será el promedio. Sin embargo, para una distribución uniforme, se debe de calcular el valor esperado de la siguiente forma: $\frac{a+b}{2}$ donde a y b son constantes numéricas.

In [5]:
data=pd.read_csv('Data_OilCompany.csv')
data.head()

,Year 1,Year 2,Year 3,Year 4,Year 5
0,203726,279969,465303,200445,96319
1,263845,272439,2192,199901,0
2,27726,334861,82265,199954,64958
3,293801,210117,275087,199934,0
4,297319,242569,464097,200598,141285


In [25]:
y1_dist=best_fit_distribution(data.iloc[:,0])
y2_dist=best_fit_distribution(data.iloc[:,1])
y3_dist=best_fit_distribution(data.iloc[:,2])
y4_dist=best_fit_distribution(data.iloc[:,3])
y5_dist=best_fit_distribution(data.iloc[:,4])

In [39]:
y1_dist, y2_dist, y3_dist,y4_dist,y5_dist

(('norm', (196593.614, 98549.90341700494)),
 ('uniform', (150096.0, 199816.0)),
 ('beta',
  (0.7832820902502415,
   0.8445188351664095,
   0.9999999999999999,
   500520.6241317627)),
 ('beta',
  (31.034327182930713,
   28.97044550413866,
   196812.8021867012,
   6161.0839821873515)),
 ('beta',
  (0.6762186672763196,
   4.754043977151072,
   -5.642702015324872e-29,
   514937.6521743612)))

In [40]:
t_dist=pd.DataFrame()
t_dist['Años']=["Year 1","Year 2","Year 3","Year 4","Year 5"]
t_dist['Distribución']=[y1_dist[0],y2_dist[0],y3_dist[0],y4_dist[0],y5_dist[0]]
t_dist['Parámetro 1']=[y1_dist[1][0],y2_dist[1][0],y3_dist[1][0],y4_dist[1][0],y5_dist[1][0]]
t_dist['Parámetro 2']=[y1_dist[1][1],y2_dist[1][1],y3_dist[1][1],y4_dist[1][1],y5_dist[1][1]]
t_dist['Parámetro 3']= ['n/a','n/a',y3_dist[1][2],y4_dist[1][2],y5_dist[1][2]]
t_dist['Parámetro 4']= ['n/a','n/a',y3_dist[1][3],y4_dist[1][3],y5_dist[1][3]]
t_dist

,Años,Distribución,Parámetro 1,Parámetro 2,Parámetro 3,Parámetro 4
0,Year 1,norm,196593.614000,98549.903417,n/a,n/a
1,Year 2,uniform,150096.000000,199816.000000,n/a,n/a
2,Year 3,beta,0.783282,0.844519,1,500521
3,Year 4,beta,31.034327,28.970446,196813,6161.08
4,Year 5,beta,0.676219,4.754044,-5.6427e-29,514938


- Valor esperado de una distribución normal, sabemos que es el promedio. 
- Valor esperado de una distribución uniforme: $\frac{a+b}{2}$

In [68]:
e_1=y1_dist[1][0]
e_1

196593.614

In [69]:
e_2=(data.iloc[:,1].max()+data.iloc[:,1].min())/2
e_2

250004.0

In [72]:
e_1+e_2

446597.614

In [73]:
5*e_1+5*e_2

2232988.0700000003

Con la propiedad del valor esperado de una variable aleatoria:

$$E[z] = E [x + y ]$$

$$= E[x]+ E [y] = E[y]+ E [x]$$ por lo tanto...

- x = Ventas Año 1
- y = Ventas Año 2
- z = Ventas en el año 1 y 2 
- P = Precio

$$E[z] = E[(P)(x)+ (P)(y)]$$

$$E[z] = E[(P)(x + y)]$$

Dado que P es una constante ....

$$E[z] = (P)E[(x + y)]$$

$$E[z] = (P)E[x] + (P)E[y]$$

Como Precio=5


$$E[z] = (5)(196593.614) + (5)(2500004)$$

$$E[z] = 2,232,988.07$$

**b) ¿Cuál es el valor esperado del retorno en valor presente para la vida del producto?**

- Primero establecemos los datos iniciales para poder obtener el valor presente.

In [9]:
rf=0.03
price=5
cost=0.8
fixed=40000
inv=1800000

In [10]:
util=data*(price-cost)-fixed
util["Year 0"]= np.ones(util.shape[0],)*(-1800000)
cols = util.columns.tolist()
cols = cols[-1:] + cols[:-1]
util=util[cols] 
util.head()

,Year 0,Year 1,Year 2,Year 3,Year 4,Year 5
0,-1800000.0,815649.2,1135869.8,1914272.6,801869.0,364539.8
1,-1800000.0,1068149.0,1104243.8,-30793.6,799584.2,-40000.0
2,-1800000.0,76449.2,1366416.2,305513.0,799806.8,232823.6
3,-1800000.0,1193964.2,842491.4,1115365.4,799722.8,-40000.0
4,-1800000.0,1208739.8,978789.8,1909207.4,802511.6,553397.0


- Fórmula de valor presente para nuestros flujos en donde:

    -Precio = P
    -Costo = C    
    -Costo Fijo = F    
    -Risk Free = 3%
    -Inv_inicial = -180,000.00

$$Vpn= -inv_{inicial}+\frac{(P-C)(Año_1)-F}{(1+0.03)^1}+\frac{(P-C)(Año_2)-F}{(1+0.03)^2}+\frac{(P-C)(Año_3)-F}{(1+0.03)^3}+\frac{(P-C)(Año_4)-F}{(1+0.03)^4}+\frac{(P-C)(Año_5)-F}{(1+0.03)^5}$$

$$E[Vpn]= E[-inv_{inicial}]+E[\frac{(P-C)(Año_1)-F}{(1+0.03)^1}]+ E[\frac{(P-C)(Año_2)-F}{(1+0.03)^2}] + E [\frac{(P-C)(Año_3)-F}{(1+0.03)^3}] + E [\frac{(P-C)(Año_4)-F}{(1+0.03)^4}] + E[\frac{(P-C)(Año_5)-F}{(1+0.03)^5}]$$


$$E[Vpn]= E[-inv_{inicial}]+\frac{E[(P-C)(Año_1)-F]}{E[(1+0.03)^1]}+ \frac{E[(P-C)(Año_2)-F]}{E[(1+0.03)^2]} +\frac{E[(P-C)(Año_3)-F]}{E[(1+0.03)^3]} + \frac{E[(P-C)(Año_4)-F]}{E[(1+0.03)^4]} + \frac{E[(P-C)(Año_5)-F]}{E[(1+0.03)^5]}$$

Como $Inv_{inicial}, (P-C), F$ y $(1+i)^n$ son constantes entonces su esperanza es la misma constante...

$$E[Vpn]= -inv_{inicial}+ \frac{(P-C)(E[Año_1])-F}{(1+0.03)^1}+ \frac{(P-C)(E[Año_2])-F}{(1+0.03)^2} +\frac{(P-C)(E[Año_3])-F}{(1+0.03)^3} + \frac{(P-C)(E[Año_4])-F}{(1+0.03)^4} + \frac{(P-C)(E[Año_5])-F}{(1+0.03)^5}$$

Cálculo de los valores esperados:

- Valor esperado de una distribución normal, sabemos que es el promedio. 
- Valor esperado de una distribución uniforme: $\frac{a+b}{2}$
- Valor esperado de una distribución lognormal: $e^{\mu+\frac{\sigma^2}{2}}$
- Valor esperado de una distribución beta: $\frac{\alpha}{\alpha+\beta}$


$$E[Vpn]= \frac{(P-C)(\mu)-F}{(1+0.03)^1}+ \frac{(P-C)(\frac{a+b}{2})-F}{(1+0.03)^2} +\frac{(P-C)(\frac{\alpha}{\alpha+\beta})-F}{(1+0.03)^3} + \frac{(P-C)(\frac{\alpha}{\alpha+\beta})-F}{(1+0.03)^4} + \frac{(P-C)(\frac{\alpha}{\alpha+\beta})-F}{(1+0.03)^5}$$

In [55]:
vp_1=util.iloc[:,1].mean()
vp_2=util.iloc[:,2].mean()
vp_3=util.iloc[:,3].mean()
vp_4=util.iloc[:,4].mean()
vp_5=util.iloc[:,5].mean()
print("Valores presentes anuales consecutivamente",vp_1,vp_2,vp_3,vp_4,vp_5,"Suma de los vp",vp_1+vp_2+vp_3+vp_4+vp_5)

Valores presentes anuales consecutivamente 785693.1788000008 1007021.8303999999 987274.1213000003 799997.0705000001 265038.55970000045 Suma de los vp 3845024.760700002


$E[Vpn]= \frac{785693.1788000008}{(1+0.03)^1}+ \frac{1007021.8303999999}{(1+0.03)^2} +\frac{987274.1213000003}{(1+0.03)^3} + \frac{799997.0705000001}{(1+0.03)^4} + \frac{265038.55970000045}{(1+0.03)^5}$

$E[Vpn]= 3,845,024.76$


**¿Cuál es la varianza?**


$$Vpn= \frac{(P-C)(Año_1)-F}{(1+0.03)^1}+\frac{(P-C)(Año_2)-F}{(1+0.03)^2}+\frac{(P-C)(Año_3)-F}{(1+0.03)^3}+\frac{(P-C)(Año_4)-F}{(1+0.03)^4}+\frac{(P-C)(Año_5)-F}{(1+0.03)^5}$$

$$VAR[Vpn]= VAR[\frac{(P-C)(Año_1)-F}{(1+0.03)^1}+\frac{(P-C)(Año_2)-F}{(1+0.03)^2}+\frac{(P-C)(Año_3)-F}{(1+0.03)^3}+\frac{(P-C)(Año_4)-F}{(1+0.03)^4}+\frac{(P-C)(Año_5)-F}{(1+0.03)^5}]$$

$$VAR[Vpn]= \frac{VAR[(P-C)(Año_1)-F]}{VAR[(1+0.03)^1]}+ \frac{VAR[(P-C)(Año_2)-F]}{VAR[(1+0.03)^2]} +\frac{VAR[(P-C)(Año_3)-F]}{VAR[(1+0.03)^3]} + \frac{VAR[(P-C)(Año_4)-F]}{VAR[(1+0.03)^4]} + \frac{VAR[(P-C)(Año_5)-F]}{VAR[(1+0.03)^5]}$$


Como (P-C), F y $(1+i)^n$ son constantes entonces salen de la varianza al cuadrado...

$$VAR[Vpn]= \frac{(P-C)^2(VAR[Año_1])-F}{((1+0.03)^1)^2}+ \frac{(P-C)^2(VAR[Año_2])-F}{((1+0.03)^2)^2} +\frac{(P-C)^2(VAR[Año_3])-F}{((1+0.03)^3)^2} + \frac{(P-C)^2(VAR[Año_4])-F}{((1+0.03)^4)^2} + \frac{(P-C)^2(VAR[Año_5])-F}{((1+0.03)^5)^2}$$

Al igual que en el valor esperado, la varianza se calcula de diferente manera para cada una de las distribuciones, por lo que:

- Varianza de una distribución normal es la desviación estandar
- Varianza de una distribución uniforme: $\frac{(b-a)^2}{12}$
- Varianza de una distribución lognormal: $(e^{\sigma ^{2}}-1)e^{2\mu +\sigma ^{2}}$
- Varianza de una distribución beta: $\frac{\alpha\beta}{(\alpha+\beta+1)(\alpha+\beta)^2}$

$$VAR[Vpn]= \frac{(P-C)^2(\sigma)}{((1+0.03)^1)^2}+ \frac{(P-C)^2(\frac{(b-a)^2}{12})}{((1+0.03)^2)^2} +\frac{(P-C)^2(\frac{\alpha\beta}{(\alpha+\beta+1)(\alpha+\beta)^2})}{((1+0.03)^3)^2} + \frac{(P-C)^2(\frac{\alpha\beta}{(\alpha+\beta+1)(\alpha+\beta)^2})}{((1+0.03)^4)^2} + \frac{(P-C)^2(\frac{\alpha\beta}{(\alpha+\beta+1)(\alpha+\beta)^2})}{((1+0.03)^5)^2}$$

In [76]:
var_1=util.iloc[:,1].var() 
var_2=util.iloc[:,2].var()
var_3=util.iloc[:,3].var()
var_4=util.iloc[:,4].var()
var_5=util.iloc[:,5].var()
var_1,var_2,var_3,var_4,var_5, 

(171406855724.01956,
 58884654110.65013,
 556212570151.7742,
 2742121.0393694313,
 101956967309.95157)

$$VAR[Vpn]= \frac{171406855724.01956}{((1+0.03)^1)^2}+ \frac{58884654110.65013}{((1+0.03)^2)^2} +\frac{556212570151.7742}{((1+0.03)^3)^2} + \frac{2742121.0393694313}{((1+0.03)^4)^2} + \frac{101956967309.95157}{((1+0.03)^5)^2}$$


$$VAR[Vpn]= 75'557,264,7985.2400$$

**c) ¿Cuál es el valor esperado del NPV y VAR del NPV para cada año?**

In [81]:
# Data frame completo de los flujos, valor pesente neto y tie

tabla = util
tie, vpn = func(0.03,util)
tabla['IRR']=tie
tabla['NPV']=vpn
tabla['NPV']=tabla['NPV'].round(2)
tabla['IRR']=tabla['IRR'].round(2)
tabla.head()

<ipython-input-21-215ba072d674>:19: DeprecationWarning: numpy.irr is deprecated and will be removed from NumPy 1.20. Use numpy_financial.irr instead (https://pypi.org/project/numpy-financial/).
  ans_irr=np.irr(flujos.iloc[i,0:6])*100
<ipython-input-21-215ba072d674>:20: DeprecationWarning: numpy.npv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.npv instead (https://pypi.org/project/numpy-financial/).
  ans_npv=np.npv(rf,flujos.iloc[i,0:6])


,Year 0,Year 1,Year 2,Year 3,Year 4,Year 5,IRR,NPV
0,-1800000.0,815649.2,1135869.8,1914272.6,801869.0,364539.8,50.81,2841294.71
1,-1800000.0,1068149.0,1104243.8,-30793.6,799584.2,-40000.0,27.23,925628.90
2,-1800000.0,76449.2,1366416.2,305513.0,799806.8,232823.6,16.70,753242.18
3,-1800000.0,1193964.2,842491.4,1115365.4,799722.8,-40000.0,43.53,1850073.83
4,-1800000.0,1208739.8,978789.8,1909207.4,802511.6,553397.0,59.10,3233718.55


Como el NPV sigue una distribución normal, podemos calcular su valor esperado de la forma convencional:


In [82]:
vpn=tabla.iloc[:,7].mean()
vpn

1754930.8720650005

Sea z el VPN

$$E[z]=1'754,930.8721101447$$

Para calcular la varianza tenemos la propiedad:

$$Var[x] = Cov(x,x)= E[x^2] - E[x]^2$$

Con nuestros datos...

$$Var[Vpn] = Cov(Vpn,Vpn)= E[Vpn^2] - E[Vpn]^2$$




In [83]:
esp_vpn=(util.iloc[:,7].mean())**2 #esperanza de vpn al cuadrado e(x)^2
esp_vpn2=((util.iloc[:,7])**2).mean()# esperanza  de vpn al cuadrado e(x^2)

var_vpn=esp_vpn2-esp_vpn
esp_vpn,esp_vpn2,var_vpn
var

(3079782365726.823, 3839594086710.42, 759811720983.5967)

$$Var[Vpn] = Cov(Vpn,Vpn)= 3839594086710.42 - 3079782365726.823$$

$$Var[Vpn] = 759,811,720,983.5967$$

$$Cov(x,y)=[\sqrt{VAR(x)*VAR(y)}][Corr(x,y)]$$


**d) Estime analíticamente la PDF del VPN y la TIR:**

1. ¿Cuál es la probabilidad de que la TIR sea superior a la tasa libre de riesgo?

In [84]:
def tabla_z(x,mu,sigma):
    res = 1-norm.cdf(x,mu,sigma)
    return res

In [85]:
tabla_z(0.03,0.3593,0.1548)

0.9833007976177081

La probabilidad de que la TIR sea superior a la tasa libre de riesgo es del **98.33%**

2. ¿Cuál es la probabilidad de que la TIR sea superior al 35%?


In [86]:
tabla_z(0.35,0.3593,0.1548)

0.5239530527241607

La probabilidad de que la TIR sea superior al 35% es del **52.3953%**

3. ¿Cuál es la probabilidad de que el valor del proyecto sea superior a 2 millones de dólares?

In [87]:
tabla_z(2000000, 1757025.24, 879278.47)

0.3911456744688774

La probabilidad de que el valor del proyecto sea superior a 2 millones de dólares es del **39.1145%**

4. ¿Cuál es la probabilidad de que la TIR esté entre el 10% y el 20%? 

In [88]:
a = tabla_z(0.10,0.3593,0.1548)
b = tabla_z(0.20,0.3593,0.1548)
print((a-b)*100,'%')

10.47626669172571 %


La probabilidad de que la TIR sea superior al 35% es del **10.4762%**

5. ¿Obtienes las mismas respuestas que en Tarea 1.1?

*Comparando las respuestas de la tarea 1.1 con las actuales se puede notar que los resultados no son exactamente los mismos, sin embargo, la diferencia es mínima ya que prácticamente varian los porcentajes en cuestión de decimales. En donde si se pudo observar una diferencia considerablemente mayor fue en el último inciso en donde la respuesta actual fue de 10.4662% y en la tarea 1.1 se ubtuvo una probabilidad del 12%.*